# Complexities

In [1]:
from collections import Counter

counter = Counter()
class CounterFloat:
    def __init__(self, v):
        self.v = v
    def __float__(self):
        return self.v
    
    def __abs__(self):
        return abs(self.v)
    def __pos__(self):
        return self
    def __neg__(self):
        return CounterFloat(-self.v)
    
    def __add__(self, other):
        counter['+'] += 1
        return CounterFloat(float(self)+float(other))
    
    def __sub__(self, other):
        counter['-'] += 1
        return CounterFloat(float(self)-float(other))
    def __rsub__(self, other): #needed for inv_gauss
        if other != 0:
            counter['-'] += 1
        return CounterFloat(other-float(self))
    
    def __mul__(self, other):
        counter['*'] += 1
        return CounterFloat(float(self)*float(other))
    def __rmul__(self, other): #needed for inv_gauss
        if other not in {-1, +1}:
            counter['*'] += 1
        return CounterFloat(float(self)*other)
    
    def __truediv__(self, other):
        if other not in {-1, +1}:
            counter['/'] += 1
        return CounterFloat(float(self)/float(other))
    def __rtruediv__(self, other): #needed for inv_gauss
        counter['/'] += 1
        return CounterFloat(other/float(self))
    
    def __repr__(self):
        return f'CounterFloat({self.v})'

## Matmul

In [2]:
import numpy as np
from linalg import matmul

for _ in range(100):
    L, N, M = np.random.randint(1, 20, size=3)
    A, B = np.random.rand(L, N), np.random.rand(N, M)
    
    counter.clear()
    prediction = matmul(np.vectorize(CounterFloat)(A), np.vectorize(CounterFloat)(B)).astype(np.float64)
    actual = A @ B
    
    assert np.allclose(prediction.astype(np.float64), actual.astype(np.float64))
    assert set(counter) <= {'+', '*'}
    assert counter['+'] == L*(N-1)*M
    assert counter['*'] == L*N*M
    assert counter.total() == L*(2*N-1)*M

## Leibniz

In [3]:
from linalg import det_leibniz
from math import factorial

for _ in range(100):
    N = np.random.randint(1, 5)
    A = np.random.rand(N, N)
    
    counter.clear()
    prediction = float(det_leibniz(np.vectorize(CounterFloat)(A)))
    actual = np.linalg.det(A)
    
    assert np.isclose(prediction, actual)
    assert set(counter) <= {'+', '*'}
    assert counter['+'] == factorial(N)-1
    assert counter['*'] == (N-1)*factorial(N)
    assert counter.total() == N*factorial(N)-1

## Gauss

In [4]:
from linalg import det_gauss

for _ in range(100):
    N = np.random.randint(1, 20)
    A = np.random.rand(N, N)
    
    counter.clear()
    prediction = float(det_gauss(np.vectorize(CounterFloat)(A)))
    actual = np.linalg.det(A)
    
    assert np.isclose(prediction, actual)
    assert set(counter) <= {'-', '*', '/'}
    assert counter['-'] == N*(N**2-1) // 3
    assert counter['*'] == N*(N**2+2) // 3 - 1
    assert counter['/'] == N*(N-1) // 2
    assert counter.total() == N*(4*N**2+3*N-1) // 6 - 1

In [5]:
from linalg import inv_gauss

for _ in range(100):
    N = np.random.randint(1, 20)
    A = np.random.rand(N, N)
    
    counter.clear()
    A_inv = inv_gauss(np.vectorize(CounterFloat)(A.copy())).astype(np.float64)
    
    assert np.allclose(A@A_inv, np.eye(N))
    assert set(counter) <= {'+', '-', '*', '/'}
    assert counter['+'] == N**3-N**2
    assert counter['-'] == 2*N**3-3*N**2+2*N-1
    assert counter['*'] == 3*N**3-3*N**2
    assert counter['/'] == 2*N**2
    assert counter.total() == 6*N**3-5*N**2+2*N-1

## Laplace

In [6]:
from linalg import det_laplace
from math import e, floor

for _ in range(100):
    N = np.random.randint(1, 7)
    A = np.random.rand(N, N)
    
    counter.clear()
    prediction = float(det_laplace(np.vectorize(CounterFloat)(A)))
    actual = np.linalg.det(A)
    
    assert np.isclose(prediction, actual)
    assert set(counter) <= {'+', '-', '*'}
    assert counter['+']+counter['-'] == factorial(N)-1
    assert counter['*'] == floor((e-1)*factorial(N))-1 #https://oeis.org/A038156
    assert counter.total() == floor(e*factorial(N))-2 #https://oeis.org/A026243

## Bareiss

In [7]:
class CounterInt:
    def __init__(self, v):
        self.v = v
    def __int__(self):
        return self.v
    
    def __abs__(self):
        return abs(self.v)
    def __pos__(self):
        return self
    def __neg__(self):
        return CounterInt(-self.v)
    
    
    def __add__(self, other):
        counter['+'] += 1
        return CounterInt(int(self)+int(other))
    def __radd__(self, other):
        if other != 0:
            counter['+'] += 1
        return CounterInt(int(self)+int(other))
    
    def __sub__(self, other):
        counter.update('-')
        return CounterInt(int(self)-int(other))
    def __rsub__(self, other):
        if other != 0:
            counter['-'] += 1
        return CounterInt(int(other)-float(self))
    
    def __mul__(self, other):
        counter['*'] += 1
        return CounterInt(int(self)*int(other))
    def __rmul__(self, other):
        if other not in {-1, +1}:
            counter['*'] += 1
        return CounterInt(int(self)*(other))
    
    def __floordiv__(self, other):
        if other not in {-1, +1}:
            counter['//'] += 1
        return CounterInt(int(self)//int(other))
    def __rfloordiv__(self, other):
        counter['//'] += 1
        return CounterInt(other//int(self))
    
    def __repr__(self):
        return f'CounterInt({self.v})'

In [8]:
from linalg import swap_pivot

def det_bareiss(A):
    s = True
    for i in range(A.shape[0]):
        #pivot
        i_max, j_max = \
                np.unravel_index(np.argmax(abs(A[i:, i:])), A[i:, i:].shape)
        #cancellation
        if not A[i+i_max, i+j_max]:
            return 0
        swap_pivot(A, i, i+i_max, i+j_max)
        s ^= bool(i_max) != bool(j_max)
        #reduction
        A[i+1:, i:] = \
                (A[i+1:, i:]*A[i, i] - A[i+1:, i][:, np.newaxis]*A[i, i:]) \
                // (A[i-1, i-1] if i>0 else 1)
    
    return +A[-1, -1] if s else -A[-1, -1]


for _ in range(100):
    N = np.random.randint(1, 10)
    A = np.random.randint(-100, +100, size=(N, N))
    
    counter.clear()
    prediction = int(det_bareiss(np.vectorize(CounterInt)(A)))
    actual = np.linalg.det(A)
    
    assert np.isclose(prediction, actual)
    assert set(counter) <= {'-', '*', '//'}
    assert counter['-'] == N*(N**2-1) // 3
    assert counter['*'] == N*(N**2-1)*2 // 3
    assert counter['//'] == N*(N**2-3*N+2) // 3
    assert counter.total() == N*(4*N**2-3*N-1) // 3

## LU

In [9]:
from linalg import LU

def is_closeperm(P):
    return np.allclose(P.sum(axis=0), 1) and np.allclose(P.sum(axis=1), 1) \
            and np.all(np.isclose(P, 1) | np.isclose(P, 0))

def is_closetril(L):
    return np.allclose(np.triu(L, 1), 0)

def is_closetriu(U):
    return np.allclose(np.tril(U, -1), 0)

for _ in range(1000):
    M, N = np.random.randint(1, 10, size=2)
    A = np.random.rand(M, N)
    
    counter.clear()
    L, U = LU(np.vectorize(CounterFloat)(A))
    L, U = L.astype(np.float64), U.astype(np.float64)
    
    assert is_closetril(L) and is_closetriu(U) and np.allclose(L@U, A)
    assert set(counter) <= {'-', '*', '/'}
    M, N = sorted((M, N))
    assert counter['-'] == M*(M-1)*N // 2
    assert counter['*'] == M*(M-1)*N // 2
    assert counter['/'] == M*(M-1) // 2
    assert counter.total() == M*(M-1)*N + M*(M-1)//2

In [10]:
from linalg import PLU

for _ in range(1000):
    M, N = np.random.randint(1, 10, size=2)
    A = np.random.rand(M, N)
    
    counter.clear()
    P, L, U = PLU(np.vectorize(CounterFloat)(A))
    P, L, U = P.astype(np.float64), L.astype(np.float64), U.astype(np.float64)
    
    assert is_closeperm(P) and is_closetril(L) and is_closetriu(U) and np.allclose(P@L@U, A)
    assert set(counter) <= {'-', '*', '/'}
    O = min(M-1, N)
    assert counter['-'] == N*O*(2*M-O-1) // 2
    assert counter['*'] == N*O*(2*M-O-1) // 2
    assert counter['/'] == O*(2*M-O-1) // 2
    assert counter.total() == N*O*(2*M-O-1) + O*(2*M-O-1)//2

In [11]:
from linalg import LUQ

for _ in range(1000):
    M, N = np.random.randint(1, 10, size=2)
    A = np.random.rand(M, N)
    
    counter.clear()
    L, U, Q = LUQ(np.vectorize(CounterFloat)(A))
    L, U, Q = L.astype(np.float64), U.astype(np.float64), Q.astype(np.float64)
    
    assert is_closetril(L) and is_closetriu(U) and is_closeperm(Q) and np.allclose(L@U@Q, A)
    assert set(counter) <= {'-', '*', '/'}
    O = min(M, N-1)
    assert counter['-'] == M*O*(2*N-O-1) // 2
    assert counter['*'] == M*O*(2*N-O-1) // 2
    assert counter['/'] == O*(2*N-O-1) // 2
    assert counter.total() == M*O*(2*N-O-1) + O*(2*N-O-1)//2

In [12]:
from linalg import PLUQ

for _ in range(1000):
    M, N = np.random.randint(1, 10, size=2)
    A = np.random.rand(M, N)
    
    counter.clear()
    P, L, U, Q = PLUQ(np.vectorize(CounterFloat)(A))
    P, L, U, Q = P.astype(np.float64), L.astype(np.float64), U.astype(np.float64), Q.astype(np.float64)
    
    assert is_closeperm(P) and is_closetril(L) and is_closetriu(U) and is_closeperm(Q) and np.allclose(P@L@U@Q, A)
    assert set(counter) <= {'-', '*', '/'}
    M, N = sorted((M, N))
    assert counter['-'] == M*(M-1)*N // 2
    assert counter['*'] == M*(M-1)*N // 2
    assert counter['/'] == M*(M-1) // 2
    assert counter.total() == M*(M-1)*N + M*(M-1)//2

## Row Echelon

In [13]:
class CounterFraction:
    def __init__(self, v):
        self.v = v
    
    def __bool__(self):
        return bool(self.v)
    
    def __abs__(self):
        return abs(self.v)
    def __pos__(self):
        return self
    def __neg__(self):
        return CounterFraction(-self.v)
    
    
    def __add__(self, other):
        if other == 0:
            print('Warning: +0')
        counter['+'] += 1
        return CounterFraction(self.v + (other.v if isinstance(other, CounterFraction) else other))
    __radd__ = __add__
    
    def __sub__(self, other):
        if other == 0:
            print('Warning: -0')
        counter.update('-')
        return CounterFraction(self.v - (other.v if isinstance(other, CounterFraction) else other))
    def __rsub__(self, other):
        return CounterFraction((other.v if isinstance(other, CounterFraction) else other) - self.v)
    
    def __mul__(self, other):
        if other in {-1, +1}:
            print('Warning: *(+-1)')
        counter['*'] += 1
        return CounterFraction(self.v * (other.v if isinstance(other, CounterFraction) else other))
    __rmul__ = __mul__
    
    def __truediv__(self, other):
        if other in {-1, +1}:
            print('Warning: /(+-1)')
        counter['/'] += 1
        return CounterFraction(self.v / (other.v if isinstance(other, CounterFraction) else other))
    def __rtruediv__(self, other):
        counter['/'] += 1
        return CounterFraction((other.v if isinstance(other, CounterFraction) else other) / self.v)
    
    def __repr__(self):
        return f'CounterFloat({self.v})'

In [14]:
from linalg import randfr, is_ref, ref_gauss

for _ in range(1000):
    M, N  = np.random.randint(1, 10, size=2)
    r = np.random.randint(0, min(M, N)+1)
    A = np.vectorize(CounterFraction)(randfr(M, N, r))
    
    counter.clear()
    ref_gauss(A, False)
    
    assert is_ref(A, False)
    assert set(counter) <= {'-', '*', '/'}
    assert counter['-'] == M*N*r - N*r*(r+1) // 2
    assert counter['*'] == M*N*r - N*r*(r+1) // 2
    assert counter['/'] == M*r - r*(r+1) // 2
    assert counter.total() == M*r*(2*N+1)-N*r*(r+1)-r*(r+1)//2